In [1]:
using HTTP
using CodecZlib
using DataFrames
using Mongoc

In [2]:
function  getDataFromURLFile(url_file)

    i = 0 ;
    while i < 5
        try 
          
            response =  HTTP.request("GET", url_file);
            body = response.body;
            filehandler = GzipDecompressorStream(IOBuffer(body));
            imdb_data =  collect(eachline(filehandler));
            
            return imdb_data;
            
            break
        catch e
            println(e);
            i += 1;
        end
    end

end

getDataFromURLFile (generic function with 1 method)

In [3]:
url_file = "https://datasets.imdbws.com/title.ratings.tsv.gz"

"https://datasets.imdbws.com/title.ratings.tsv.gz"

In [4]:
imdb_data  = getDataFromURLFile(url_file);

In [5]:
 #elimina el primer elemento del vector
imdb_data = [split(x, "\t") for x in imdb_data];
header = imdb_data[1];
deleteat!(imdb_data, 1);

In [6]:
mongo_data = Vector{Dict}()
for data in imdb_data
    subdata_mongo = Dict()
    for (key, value) in zip(header, data)
        println(key, value)
        if occursin(r"^[\d(.)?]+$", value)
            value = parse.(Float64,value)
        end
        subdata_mongo[key] = value
        
    end
    println(subdata_mongo)
    push!(mongo_data, subdata_mongo)
    println(mongo_data)
end


tconsttt0000001
averageRating5.7
numVotes2055
Dict{Any, Any}("tconst" => "tt0000001", "averageRating" => 5.7, "numVotes" => 2055.0)
Dict[Dict{Any, Any}("tconst" => "tt0000001", "averageRating" => 5.7, "numVotes" => 2055.0)]
